In [ ]:
#!/usr/bin/env python
# coding: utf-8
# import utils
# import helper_functions # import extract_attributes_from_tokens,token_attributes_from_sentence, count_adjectives_in_document,get_entities,get_digits,add_spacy_pipe,add_spacy_macher,add_spacy_macher_view
import spacy
# from spacy import displacy 
from spacy.matcher import Matcher
import pandas as pd
import unicodedata
from unidecode import unidecode
from Levenshtein import *
from collections import defaultdict
import textdescriptives as td  #TEXTDESCRIPTIVES FR PSACY3
import numpy as np
import time
# Record the start time
start_time = time.time()


# rows_to_show = 5 we update this below after we read the news feed ...
#Load lib el
# nlp = spacy.load("el_core_news_sm")
# nlp = spacy.load("el_core_news_md")

nlp = spacy.load("el_core_news_lg")
matcher = Matcher(nlp.vocab)

def remove_greek_tones(input_string):
    return ''.join(
        char for char in unicodedata.normalize('NFD', input_string)
        if unicodedata.category(char) != 'Mn'
    )

def custom_filter(token):
   custom_stopwords = ['ακομα','ακομη','αλλα','αλλες','αλλη','αλλης','αλλων','αν','ανα','αναμεσα','απ','απο','αρα','αραγε','αργα','αρκετα','ας','αυριο','αυτα','αυτες','αυτη','αυτην','αυτης','αυτο','αυτοι','αυτον','αυτος','αυτου','αυτους','αυτων','αφου','γι','για','γιατι','δα','δε','δει','δεινα','δεν','δηλαδη','δι','δια','διοτι','δις','διχως','εαν','ειδεμη','ειθε','ειπαν','ειπε','εις','ειτε','εκαστα','εκαστες','εκαστη','εκαστην','εκαστης','εκαστο','εκαστοι','εκαστον','εκαστος','εκαστου','εκαστους','εκαστων','εκει','εκεινα','εκεινες','εκεινη','εκεινην','εκεινης','εκεινο','εκεινοι','εκεινον','εκεινος','εκεινου','εκεινους','εκεινων','εκτος','εμας','εμεις','εμενα','εμπρος','εν','ενα','εναν','ενας','ενος','εντελως','εντος','εντωμεταξυ','ενω','ενωπιον','εξ','εξαφνα','εξης','εξισου','εξω','επανω','επειδη','επειτα','επι','επιπλεον','επισης','επομενως','εσας','εσεις','εσενα','εστω','εσυ','ετερα','ετεραι','ετερας','ετερες','ετερη','ετερης','ετερο','ετεροι','ετερον','ετερος','ετερου','ετερους','ετερων','ετουτα','ετουτες','ετουτη','ετουτην','ετουτης','ετουτο','ετουτοι','ετουτον','ετουτος','ετουτου','ετουτους','ετουτων','ετσι','ευθυς','εφεξης','εχθες','εχτες','εχω','εως','η','ηδη','ητοι','ηττον','θα','ιδια','ιδιαν','ιδιας','ιδιες','ιδιο','ιδιοι','ιδιον','ιδιος','ιδιου','ιδιους','ιδιων','ιδιως','ισαμε','ισια','ισως','καθε','καθεμια','καθεμιας','καθενα','καθενας','καθενος','καθετι','καθολου','καθως','και','καμια','καμιαν','καμιας','καν','κανεν','κανενα','κανεναν','κανενας','κανενος','κανετε','κανουμε','κανουν','καποια','καποιαν','καποιας','καποιες','καποιο','καποιοι','καποιον','καποιος','καποιου','καποιους','καποιων','καποτε','καπου','καπως','κατ','κατι','κατιτι','κατοπιν','κατω','κι','κιολας','κλπ','κοντα','κτλ','κυριως','λογω','λοιπα','λοιπον','μα','μας','μαζι','μακαρι','μαλιστα','μαλλον','μας','με','μεθαυριο','μεμιας','μεν','μερικα','μερικες','μερικοι','μερικους','μερικων','μεσα','μεσω','μετ','μετα','μεταξυ','μεχρι','μη','μηδε','μην','μηπως','μητε','μια','μιαν','μιας','μολις','μολονοτι','μονομιας','μου','να','ναι','ο','οι','ολα','ολες','ολη','ολην','ολης','ολο','ολοι','ολον','ολονεν','ολος','ολου','ολους','ολων','ολως','ολωσδιολου','ομως','οποια','οποιαδηποτε','οποιαν','οποιανδηποτε','οποιας','οποιασδηποτε','οποιδηποτε','οποιες','οποιεσδηποτε','οποιο','οποιοδηποτε','οποιοι','οποιον','οποιονδηποτε','οποιος','οποιοσδηποτε','οποιου','οποιουδηποτε','οποιους','οποιουσδηποτε','οποιων','οποιωνδηποτε','οποτε','οποτεδηποτε','οπου','οπουδηποτε','οπως','ορισμενα','ορισμενες','ορισμενων','ορισμενως','οσα','οσαδηποτε','οσες','οσεσδηποτε','οση','οσηδηποτε','οσην','οσηνδηποτε','οσης','οσησδηποτε','οσο','οσοδηποτε','οσοι','οσοιδηποτε','οσον','οσονδηποτε','οσος','οσοσδηποτε','οσου','οσουδηποτε','οσους','οσουσδηποτε','οσων','οσωνδηποτε','οταν','οτι','οτιδηποτε','οτου','ου','ουδε','ουτε','παντα','παντοτε','παντου','παντως','παρ','παρα','πει','περα','περι','περιπου','περσι','περυσι','πια','πιθανον','πιο','πισω','πλαι','πλεον','πλην','ποια','ποιαν','ποιας','ποιες','ποιο','ποιοι','ποιον','ποιος','ποιου','ποιους','ποιων','πολλα','πολυ','ποσες','ποση','ποσην','ποσης','ποσο','ποσοι','ποσος','ποσους','ποτε','που','πουθε','πουθενα','πρεπει','πριν','προ','προκειται','προς','προτου','προχθες','προχτες','πρωτα','πρωτη','πρωτης','πρωτυτερα','πως','σαν','σας','σε','σεις','σημερα','σιγα','σου','στα','στη','στην','στης','στις','στο','στον','στου','στους','στων','συν','συχνα','συχνας','συχνες','συχνη','συχνην','συχνης','συχνο','συχνοι','συχνον','συχνος','συχνου','συχνου','συχνους','συχνων','συχνως','σχεδον','σωστα','τα','ταδε','ταυτα','ταυτες','ταυτη','ταυτην','ταυτης','ταυτο','ταυτον','ταυτος','ταυτου','ταυτοχρονα','ταυτων','τελικα','τελικως','τες','τετοια','τετοιαν','τετοιας','τετοιες','τετοιο','τετοιοι','τετοιον','τετοιος','τετοιου','τετοιους','τετοιων','τη','την','της','τι','τιποτα','τιποτε','τις','το','τοι','τον','τος','τοσα','τοσες','τοση','τοσην','τοσης','τοσο','τοσοι','τοσον','τοσος','τοσου','τοσους','τοσων','τοτε','του','τουλαχιστο','τουλαχιστον','τους','τουτα','τουτες','τουτη','τουτην','τουτης','τουτο','τουτοι','τουτοις','τουτον','τουτος','τουτου','τουτους','τουτων','τυχον','των','τωρα','υπ','υπερ','υπο','υποψη','υποψιν','υστερα','χθες','χτες','χωρις','ψηλα','ω','ως','ωσαν','ωσοτου','ωσπου','ωστε','ωστοσο','ωχ']
   remove_words = ['νυχτερινό','νυχτερινό','νυχτερινό','νυχτερινό','παρακάτω','τουρκικά','εργαζόμενοι','παράσχει','στοίχισε','μέρες','είδε','σκούρο','ανθρωποκυνητό','κεφαλαίου','ανάκτορα','μέτρων','δίνει','πούμε','έκλεισε','κομμάτων','δεκάδες','χαράτσι','Ασφάλισης','υπολογιστεί','υπολογιστούν','πλήρωνε','παραμείνει','αντιπρόεδρος','έδωσε','Πατρινός','αντιπάλων','εξωτερικό','πελάγη','νομού','Στερεάς','περίπτωση','οργανωτές','σωματεία','παράταξή','Παράταξης','πάνε','μεταβούν','απαντηθούν','παραστεί','καταληκτική','ερωτηθεί','χαρτών','σπουδαστές','επιτάχθηκε','κατάλοιπο','στάτους','προσωπικά','μεγάρου','κανείς','παλαιά','καράβια','πλάνα','πλάνα','αφορά','κρίνει','επισκεφτεί','ακούσει','φυλακών','Περιστύλιο','στον','εστίαση','πλήξει','κοινοποιήσεις','ανάδειξη','φόρες','τροχαίου','έμφραγμα','παράσημα','φορέσω','βάλω','παράτυπη','προς','είδες','παραχθεί','πρωτεύουσα','αυτή','σύμφωνα','εργαστούμε','Μέση','κανέναν','αυτό','εκατ.','επικεφαλής','κλιμακίων','πάνω','παραίνεση','μισή','κινητήρια','πάντα','Αναστασίου','παραστάσεις','Σύμφωνα','διάσπαρτες','όλων','Διάσελλα','ποταμό','ποταμού','παραδόθηκε','συντάξεων','μέσω','προσφύγων','ανακατανομή','κατάφερε','επιδείξει','λόγων','πουν','ψηφίσει','ψηφίσει','ψηφίσει','κατάλαβε','κάτι','πάλι','παραπάνω','πρωτόδικη','παραπέμπει','προέβη','κυκλώματα','παραπέμπουν','περιπτώσεις','καμία','ασκό','δώσει','άλλους','πιάτσα','αυτά','συμβολαίου','παγίου','παγίου','καταστήματα','εστίασε','επικεφαλής','κατοικίδια','κατοικίδιων','εκδίκαση','ούτε','αφορμή','γκέτο','Αντιπρόεδρος','αυτονόητα','ακούσω','μακάρι','προσωπείο','ανέχεστε','φέρετε','περιέλθει','σούπερ','κούρεμα','βόμβες','καλά','καλά','κάθεσαι','γκολ','Περόν','μπλοκάκια','σκληρόδετη','ποτέ','παράκανες','καταστήσουν','φορέσει','Συνεπώς','οικισμό','επίδοξων','κατάσβεσή','επικράτεια','Πικράκη','παράδεισο','απόκρισης','επιδόσεις','στου','Σέιχ','παρασκήνιο','ζήτημα','παράγει','πλακέτα','νομο','στίβο','κατάλαβα','παράγει','ψηφίζει','Μαργαρίτης','δουλικά','σταυρόλεξα','δουλικά','δουλικά','δουλικά','αφορμές','απλόχερα','τέλει','κομματικών','Πανούσης','τριάντα','είδα','ζήτησα','κράτησε','Καταρχάς','προβληθεί','Παρόλα','μπλογκ','δεσπόζει','επίγειος','παράδεισος','μπλογκ','κατευθύνει','πλάτη','επιτρόπου','παλαιό','περίπτωσή','κουρέψει','κανένα','βάλε','Πάνω','Είδα','παρόντες','παρόντες','μένουν','αντιπρόεδρο','κράτη','καντήλι','καταπιούν','ποταμός','γκάμα','ποστ','τετράπτυχο','site','Κρανίδι','αποστρατεία','κουμπάρα','κατήφορο','επιμέρους','παλαιάς','συνθήματα','παρότι','άλλοτε','σταθεί','μεσαίωνα','πηγε','δυνάμεων','τετράποδων','γέννα','πατάτας','πετάξω','προσφυγικές','κοίλης','στοπ','αναμένοντας','κατάπτυστες','Παρασκευάς','πλαστικό','χωρίσαμε','βρίσκουμε','αγοράκι','φορούσε','γενεές','μέτρου','χαρεί','σοκάκια','πανάκριβη','μάγια','έκτοτε','συνοδό','νυχτερινή','γκουρμέ','καύσωνα','φοράνε','ανοιχτόχρωμα','κοινοτήτων','τελείται','μασάς','περιφρούρησης','εισόδους','περιφρούρησης','αναπάντητα','υποκατάστημα','έκλεισαν','στηθεί','σοροί','επιβατών','προβλήματος','καραμπίνα','συμβολαίων','επισκεπτών','κουταβιών','ντου','διαδικτύου','διαδικτύου','κατηγορούσαν','υπέρογκη','υπέρογκα','κάμπινγκ','αναπάντητη','αποσταλεί','παραπόνου','αντίδικοι','παράξει','προκατόχους','συγκρίνει','σκανδάλων','κομματικό','πετάμε','κατάσβεση','θηλυ','επιχειρήσουν','κρίθηκε','κρούσματα','Παρασκευής','κατακαίει','αντιπάλους','πλήρωσε','λογισμικό','στενάχωρη','ασφάλισης','μέρας','Φυσικής','δεκάδων','διαμέτρου','τακτά','κομματικούς','κομματική','άνοιξη','σχολικής','βραδιά','γκέλα','ουραγό','ξαναπαίξει','συμφωνική','φορούσαν','Σωματείο','μπλακ','Προς','τουρνικέ','παρατάξεις','τραπεζιών','καταστήσει','κατάφεραν','κανένας','συνεργεία','συνεργείου','ακούσεις','ακούσεις','πατάνε','προτέρων','παραλίγο','τρίποντα','καρτών','μαστίχα','συμπεριέλαβε','μισό','μαστίχας','διατρέχει','συμπεριέλαβε','επίγειες','εντύπων','τετράμηνο','εγκαίρως','μικρόσωμα','μάγδα','μάγδα','συμμετείχε','καταλανικού','καταλανική','μωρέ','κονσερβοκουτι','μοίρασαν','αριστερας','σημερα','πρότερη','αυτοκριτική','διάφανα','Επικεφαλής','καραβάκι','βαρύτατο','κρέμασε','κρέμασε','μυθιστορήματος','αντιστοιχεί','Ποτέ','στεφάνου','επισκεφθεί','βασίσει','επισκεφθεί','Προσφύγων','Κάρολε','κρίνονται','πατάτες','πατάτες','μνήμα','μνήματα','τετράποδου','στεφάνων','ομότιτλο','φαντάρων','νυχτερινού','αδιάβλητο','καμπάνα','αδιάβλητο','Ανδρικάκη','περιβόητη','περιβόητη','περιβόητη','τετράποδοι','τόλμησαν','τόλμησαν','παλαιοί','κρατήσαμε','καταπάνω','συγκληθεί','διαδραση','διάβαζε','διάβαζα','διάβαζαν','Φυσική','Φυσικό','άνοδο','άνοδο','Εξάλλου','Κάμπου','φιλμ','φιλμ','φιλμ','φιλμ','καλούσες','κλόου','μπακ','δημοσιοποίησε','πέταξε','παντειο','καταβάσεις','κάθεται','Μέρες','παντελης','παράταξή','προβλήματος','επισκεφθεί','ζήτημα','ίδρυσε','έπαθαν','επιδόθηκε','αντιπάλου','προσεχή','κρίνουν','προβλήματά','επιλέξει','μέριμνα','καλούμε','εκατ','πρωτεύουσας','κομματικά','φαρμάκων','φαρμάκου','απέλυσε','προστατευόμενων','μαρτύρων','περιστύλιο','διάβασε','προστατευμένοι','χρονιάς','πετάνε','πολίτικη','ανατροφοδοτεί','καλούνται','πρωτογενή','ακριβών','διαφανεί','μαντάρα','Μπούλες','μοίρασε','αποκριάς','πολιτιστικούς','καλούμπα','Σφίγγει','αποκαλούμε','μαντάτα','σκόρερ','στεριάς','δίκτο','σοκάκι','καταστεί','διαβρώσει','σπείρεις','τελεί','κατάληψη','καθαρισμό','μαρίνα','παρών','κλιμάκια','προσφυγιάς','πλήρωσαν','εξάρθρωσε','εξάρθρωσε','παρατάσεις','πόρταλ','τόνισε','παραπανω','ακριβή','έκλεισα','κατάρτιση','λάβαμε','παραδοτέα','Επικρατείας','προβούν','πείσει','παραδόσεις','σπάσουν','χωριών','αντιπαροχή','Φυσικών','πολυτέκνων','αποδομήσω','προβλήθηκε','αποδομεί','Καθημερινής','Εσωτερικών','βαρειά','παραφράσει','ανακρίβεια','κρατήσει','παράταξης','μπαράζ','επικρίνει','πρόσφεραν','συνέλεξαν','μπουτόν','σκούρα','ανοιχτόχρωμο','εγκληματολόγος','εγκληματολόγο','περιστασιακή','παραλόγου','προγραμματισμένων','προβούμε','συνεργείο','λάβαρα','περσινό','συλληψή','σκιών','Παράταξη','χρίσμα','Συμμετείχαν','συμβάντων','ανεύρεση','σορός','μπάζα','ανακρίβειες','φαρμακερή','φαρμακερή','ασφάλιση','κατάπτυστος','κατάπτυστος','τετράποδος','μπαλάκια','πατριαρχείου','Πεντάγωνο','μήνυσης','προβλήµατα','προκαταβολική','κάρτ','νεύρα','φράγκο','ομολογιακό','ομολογιακού','μετρά','νομό','χρίσµα','Μαντάς','πλάτες','ρόδο','αποτέλεσαν','αποτέλεσαν','μήνυση','γυναικούλα','περσινή','νυχτερινής','δίνω','line','line','αντίποδα','Κομματικής','περιβόητου','περίδο','ακούσετε','ανήμερα','στημένη','στημένη','κομματικής','τόνιζε','μποι','Κόλλια','Παρασκάκης','σκάει','έκτροπα','επαναλάβει','Μπανάλ','αντιδημάρχου','κόλπους','κοντράρει','πρόσημο','κρατήσουν','καταληψιών','Πετσίτη','Πετσίτη','Ούτε','λιβάνι','Ποτάμηδες','ανάποδο','διορίες','φοράει','καταρράκωνε','τοσο','τοσο','κομματικοί','Περιστέρα','βαρελότα','προτού','δουλεια','κουτάκια','κουτάκια','προσέξτε','σκαλπ','σκαλπ','ζευγάς','σωματείο','μίλα','μούφα','εξοργισμένους','παράταξη','μπας','αρμάτων','αρμάτων','αρμάτων','παραδόθηκαν','Παράδεισο','Παράδεισο','απόλυτη','Δημοκράτες-66','ελιτ','καταρρεύσει','κομματίδια','κομματίδια','παρέλαβε','καμπανάκια','καμπάνες','μπούκαρε','αγωνιστής','τρέχα','προφυλαχθούν','υπουργάκου','κομματικού','στρατάρχη','στρατάρχη','μπουκάρουν','μπουκάρουν','περσινά','χωρίου','χωρίον','κατάφερες','κουμπάρες','γεράκος','πουλάει','πατάτα','χαλάσει','μπαλάκι','μπαλάκι','τουμπεκί','φαντασμάτων','Συμμετείχε','επικριτές','ψηφίζω','περίπολος','περίπολο','χωρίας','φυλακές','πολεμήσει','τερματίσει','χάρηκα','νόμιζες','γραμμένος','συνετέθη','χωρίστηκαν','επανακτήσει','ονομαστής','προφυλάκισης','καταλάβει','χαλίφη','αναστάσιμο','Ελευθέριος','προγραμματισμένες','προγραμματισμένη','τετράμηνη','προγραμματισμένο','αναπνευστικά','εξυπηρετούμενα','ανταλλάγματα','διευθύνοντος','συντηρημένες','σχηματισμένο','πλαγιές','υποδοχή','επισκεφθώ','προγραμματιστεί','χαρακτηριστεί','μπουλινγκ','κατάλαβες','μπόρα','επιχειρησιακή','οικονομολόγος','περίοπτη','σύμπλευση','δικαιώματα','συσχετισθεί','αστρονομικό','τροχός','πνευμόνια','κρεμάλες','κεφαλικό','καταλήψεις','κοινοτικές','μισά','συνεδριακό','πρωτοφανές','Ενοχος','αδιαφανές','φαντάζει','χρηματισμού','para','φαντάσματα','φίλες','πλευρό','επιμεληθεί','πρόσθεσε','επιτραπεί','οικισμούς','έπρεπε','μπάρα','πλαστικά','συσκευασίες','συσκευασία','συσκευασιών','σχεδίου','τραπεζικό','μέγεθός','σφουγγαράκια','επιχείρησαν','παράλια','συνεχίζει','επιχειρούσαν','προσθέτει','very','παρεκκλίσεις','πατάτε','διαρκές','κεράσουν','υποδοχής','συσχετισμός','υπόθεση.της','τροπή','εφόδου'] 

   remove_words_no_tones = [remove_greek_tones(word) for word in remove_words]
   # print(remove_words_no_tones[1:10])
   custom_stopwords = custom_stopwords + remove_words_no_tones
   #check if stop words exists in the token.text with not tones 
   remove_words_unique = list(set(custom_stopwords))
   token_no_tone = (''.join([c for c in unicodedata.normalize('NFD', token.text) if not unicodedata.combining(c)]))

   if token_no_tone in remove_words_unique:
       return not(True)
   return not (
         token.is_punct
       or str(token.dep_) == "advmod"
       or str(token.dep_) == "case"
       or str(token.dep_) == "cc"
       or str(token.dep_) == "dobj"
       or str(token.dep_) == "dep"
       or str(token.dep_) == "det"
       or str(token.dep_) == "mark"
       or str(token.dep_) == "pobj"
       or str(token.dep_) == "pron"
       # or str(token.dep_) == "punct"
       or str(token.dep_) == "sconj"
       or str(token.tag_) == "ADP"
       or str(token.tag_) == "ADV"
       or str(token.tag_) == "CCONJ"
       or str(token.tag_) == "DET"
       or str(token.tag_) == "PRON"
       or str(token.tag_) == "PART"
       or str(token.tag_) == "PUNCT"
       or str(token.tag_) == "PROPN"
       or str(token.tag_) == "SCONJ"
       or str(token.tag_) == "SPACE"
       or token.is_digit
       or token.is_stop
   )

def define_cleantext(text):
    #just for debugging time return 
#     return(text,text)
    doc_content = nlp(text)
    filtered_tokens_with_duplicates_content = [
        (token.text,token.lemma_) for token in doc_content if custom_filter(token)
    ]
    # print(filtered_tokens_with_duplicates_content)
    original_tokens = [token[0] for token in filtered_tokens_with_duplicates_content]
    lemmatized_tokens = [token[1] for token in filtered_tokens_with_duplicates_content]
    tokens_content = ' '.join(original_tokens)
    lemmatized_tokens_content = ' '.join(lemmatized_tokens)
    # print(tokens_content)
    # print(lemmatized_tokens_content)
    return(tokens_content,lemmatized_tokens_content)

def valence(column):
    words = column.split()
    greek_words = set(words) & set(df['Greek'].tolist())

def define_valence(column):
    words = column.split()

    valence = 0.5
    arousal = 0.5
    dominance = 0.5
    matched_tokens = 0

    for word in words:
        found_exact = False
        if word in indexes:
            found_exact = True
            indx = indexes[word]

            valence = (valence + df.at[indx, "Valence"]) / 2
            arousal = (arousal + df.at[indx, "Arousal"]) / 2
            dominance = (dominance + df.at[indx, "Dominance"]) / 2

            matched_tokens+= 1
    try:
        return (valence, arousal,dominance)

    except:
        return False


df_news = pd.read_csv("../NEW_DATA/PENDING/nafteboriki_ellada_fb.csv")

# Get the total number of rows using shape // use it to run for all rows 
total_rows = df_news.shape[0]
# rows_to_show = 10 #//update accordingly
rows_to_show = total_rows #//update accordingly



# df = pd.read_csv("../LEXIKA/fixed_greek_NRC_VAD_lex_2023_01_sorted_no_english.csv")
df = pd.read_csv("../LEXIKA/fixed_greek_EMOLEX_2023_01_sorted_no_english.csv")
indexes = {}
df = df.fillna("N/A")
for index, row in df.iterrows():
    df.at[index, "Greek"] = row["Greek"]
    indexes[df.at[index, "Greek"]] = index


custom_stopwords = ['ακομα','ακομη','αλλα','αλλες','αλλη','αλλης','αλλων','αν','ανα','αναμεσα','απ','απο','αρα','αραγε','αργα','αρκετα','ας','αυριο','αυτα','αυτες','αυτη','αυτην','αυτης','αυτο','αυτοι','αυτον','αυτος','αυτου','αυτους','αυτων','αφου','γι','για','γιατι','δα','δε','δει','δεινα','δεν','δηλαδη','δι','δια','διοτι','δις','διχως','εαν','ειδεμη','ειθε','ειπαν','ειπε','εις','ειτε','εκαστα','εκαστες','εκαστη','εκαστην','εκαστης','εκαστο','εκαστοι','εκαστον','εκαστος','εκαστου','εκαστους','εκαστων','εκει','εκεινα','εκεινες','εκεινη','εκεινην','εκεινης','εκεινο','εκεινοι','εκεινον','εκεινος','εκεινου','εκεινους','εκεινων','εκτος','εμας','εμεις','εμενα','εμπρος','εν','ενα','εναν','ενας','ενος','εντελως','εντος','εντωμεταξυ','ενω','ενωπιον','εξ','εξαφνα','εξης','εξισου','εξω','επανω','επειδη','επειτα','επι','επιπλεον','επισης','επομενως','εσας','εσεις','εσενα','εστω','εσυ','ετερα','ετεραι','ετερας','ετερες','ετερη','ετερης','ετερο','ετεροι','ετερον','ετερος','ετερου','ετερους','ετερων','ετουτα','ετουτες','ετουτη','ετουτην','ετουτης','ετουτο','ετουτοι','ετουτον','ετουτος','ετουτου','ετουτους','ετουτων','ετσι','ευθυς','εφεξης','εχθες','εχτες','εχω','εως','η','ηδη','ητοι','ηττον','θα','ιδια','ιδιαν','ιδιας','ιδιες','ιδιο','ιδιοι','ιδιον','ιδιος','ιδιου','ιδιους','ιδιων','ιδιως','ισαμε','ισια','ισως','καθε','καθεμια','καθεμιας','καθενα','καθενας','καθενος','καθετι','καθολου','καθως','και','καμια','καμιαν','καμιας','καν','κανεν','κανενα','κανεναν','κανενας','κανενος','κανετε','κανουμε','κανουν','καποια','καποιαν','καποιας','καποιες','καποιο','καποιοι','καποιον','καποιος','καποιου','καποιους','καποιων','καποτε','καπου','καπως','κατ','κατι','κατιτι','κατοπιν','κατω','κι','κιολας','κλπ','κοντα','κτλ','κυριως','λογω','λοιπα','λοιπον','μα','μας','μαζι','μακαρι','μαλιστα','μαλλον','μας','με','μεθαυριο','μεμιας','μεν','μερικα','μερικες','μερικοι','μερικους','μερικων','μεσα','μεσω','μετ','μετα','μεταξυ','μεχρι','μη','μηδε','μην','μηπως','μητε','μια','μιαν','μιας','μολις','μολονοτι','μονομιας','μου','να','ναι','ο','οι','ολα','ολες','ολη','ολην','ολης','ολο','ολοι','ολον','ολονεν','ολος','ολου','ολους','ολων','ολως','ολωσδιολου','ομως','οποια','οποιαδηποτε','οποιαν','οποιανδηποτε','οποιας','οποιασδηποτε','οποιδηποτε','οποιες','οποιεσδηποτε','οποιο','οποιοδηποτε','οποιοι','οποιον','οποιονδηποτε','οποιος','οποιοσδηποτε','οποιου','οποιουδηποτε','οποιους','οποιουσδηποτε','οποιων','οποιωνδηποτε','οποτε','οποτεδηποτε','οπου','οπουδηποτε','οπως','ορισμενα','ορισμενες','ορισμενων','ορισμενως','οσα','οσαδηποτε','οσες','οσεσδηποτε','οση','οσηδηποτε','οσην','οσηνδηποτε','οσης','οσησδηποτε','οσο','οσοδηποτε','οσοι','οσοιδηποτε','οσον','οσονδηποτε','οσος','οσοσδηποτε','οσου','οσουδηποτε','οσους','οσουσδηποτε','οσων','οσωνδηποτε','οταν','οτι','οτιδηποτε','οτου','ου','ουδε','ουτε','παντα','παντοτε','παντου','παντως','παρ','παρα','πει','περα','περι','περιπου','περσι','περυσι','πια','πιθανον','πιο','πισω','πλαι','πλεον','πλην','ποια','ποιαν','ποιας','ποιες','ποιο','ποιοι','ποιον','ποιος','ποιου','ποιους','ποιων','πολλα','πολυ','ποσες','ποση','ποσην','ποσης','ποσο','ποσοι','ποσος','ποσους','ποτε','που','πουθε','πουθενα','πρεπει','πριν','προ','προκειται','προς','προτου','προχθες','προχτες','πρωτα','πρωτη','πρωτης','πρωτυτερα','πως','σαν','σας','σε','σεις','σημερα','σιγα','σου','στα','στη','στην','στης','στις','στο','στον','στου','στους','στων','συν','συχνα','συχνας','συχνες','συχνη','συχνην','συχνης','συχνο','συχνοι','συχνον','συχνος','συχνου','συχνου','συχνους','συχνων','συχνως','σχεδον','σωστα','τα','ταδε','ταυτα','ταυτες','ταυτη','ταυτην','ταυτης','ταυτο','ταυτον','ταυτος','ταυτου','ταυτοχρονα','ταυτων','τελικα','τελικως','τες','τετοια','τετοιαν','τετοιας','τετοιες','τετοιο','τετοιοι','τετοιον','τετοιος','τετοιου','τετοιους','τετοιων','τη','την','της','τι','τιποτα','τιποτε','τις','το','τοι','τον','τος','τοσα','τοσες','τοση','τοσην','τοσης','τοσο','τοσοι','τοσον','τοσος','τοσου','τοσους','τοσων','τοτε','του','τουλαχιστο','τουλαχιστον','τους','τουτα','τουτες','τουτη','τουτην','τουτης','τουτο','τουτοι','τουτοις','τουτον','τουτος','τουτου','τουτους','τουτων','τυχον','των','τωρα','υπ','υπερ','υπο','υποψη','υποψιν','υστερα','χθες','χτες','χωρις','ψηλα','ω','ως','ωσαν','ωσοτου','ωσπου','ωστε','ωστοσο','ωχ']
remove_words = ['νυχτερινό','νυχτερινό','νυχτερινό','νυχτερινό','παρακάτω','τουρκικά','εργαζόμενοι','παράσχει','στοίχισε','μέρες','είδε','σκούρο','ανθρωποκυνητό','κεφαλαίου','ανάκτορα','μέτρων','δίνει','πούμε','έκλεισε','κομμάτων','δεκάδες','χαράτσι','Ασφάλισης','υπολογιστεί','υπολογιστούν','πλήρωνε','παραμείνει','αντιπρόεδρος','έδωσε','Πατρινός','αντιπάλων','εξωτερικό','πελάγη','νομού','Στερεάς','περίπτωση','οργανωτές','σωματεία','παράταξή','Παράταξης','πάνε','μεταβούν','απαντηθούν','παραστεί','καταληκτική','ερωτηθεί','χαρτών','σπουδαστές','επιτάχθηκε','κατάλοιπο','στάτους','προσωπικά','μεγάρου','κανείς','παλαιά','καράβια','πλάνα','πλάνα','αφορά','κρίνει','επισκεφτεί','ακούσει','φυλακών','Περιστύλιο','στον','εστίαση','πλήξει','κοινοποιήσεις','ανάδειξη','φόρες','τροχαίου','έμφραγμα','παράσημα','φορέσω','βάλω','παράτυπη','προς','είδες','παραχθεί','πρωτεύουσα','αυτή','σύμφωνα','εργαστούμε','Μέση','κανέναν','αυτό','εκατ.','επικεφαλής','κλιμακίων','πάνω','παραίνεση','μισή','κινητήρια','πάντα','Αναστασίου','παραστάσεις','Σύμφωνα','διάσπαρτες','όλων','Διάσελλα','ποταμό','ποταμού','παραδόθηκε','συντάξεων','μέσω','προσφύγων','ανακατανομή','κατάφερε','επιδείξει','λόγων','πουν','ψηφίσει','ψηφίσει','ψηφίσει','κατάλαβε','κάτι','πάλι','παραπάνω','πρωτόδικη','παραπέμπει','προέβη','κυκλώματα','παραπέμπουν','περιπτώσεις','καμία','ασκό','δώσει','άλλους','πιάτσα','αυτά','συμβολαίου','παγίου','παγίου','καταστήματα','εστίασε','επικεφαλής','κατοικίδια','κατοικίδιων','εκδίκαση','ούτε','αφορμή','γκέτο','Αντιπρόεδρος','αυτονόητα','ακούσω','μακάρι','προσωπείο','ανέχεστε','φέρετε','περιέλθει','σούπερ','κούρεμα','βόμβες','καλά','καλά','κάθεσαι','γκολ','Περόν','μπλοκάκια','σκληρόδετη','ποτέ','παράκανες','καταστήσουν','φορέσει','Συνεπώς','οικισμό','επίδοξων','κατάσβεσή','επικράτεια','Πικράκη','παράδεισο','απόκρισης','επιδόσεις','στου','Σέιχ','παρασκήνιο','ζήτημα','παράγει','πλακέτα','νομο','στίβο','κατάλαβα','παράγει','ψηφίζει','Μαργαρίτης','δουλικά','σταυρόλεξα','δουλικά','δουλικά','δουλικά','αφορμές','απλόχερα','τέλει','κομματικών','Πανούσης','τριάντα','είδα','ζήτησα','κράτησε','Καταρχάς','προβληθεί','Παρόλα','μπλογκ','δεσπόζει','επίγειος','παράδεισος','μπλογκ','κατευθύνει','πλάτη','επιτρόπου','παλαιό','περίπτωσή','κουρέψει','κανένα','βάλε','Πάνω','Είδα','παρόντες','παρόντες','μένουν','αντιπρόεδρο','κράτη','καντήλι','καταπιούν','ποταμός','γκάμα','ποστ','τετράπτυχο','site','Κρανίδι','αποστρατεία','κουμπάρα','κατήφορο','επιμέρους','παλαιάς','συνθήματα','παρότι','άλλοτε','σταθεί','μεσαίωνα','πηγε','δυνάμεων','τετράποδων','γέννα','πατάτας','πετάξω','προσφυγικές','κοίλης','στοπ','αναμένοντας','κατάπτυστες','Παρασκευάς','πλαστικό','χωρίσαμε','βρίσκουμε','αγοράκι','φορούσε','γενεές','μέτρου','χαρεί','σοκάκια','πανάκριβη','μάγια','έκτοτε','συνοδό','νυχτερινή','γκουρμέ','καύσωνα','φοράνε','ανοιχτόχρωμα','κοινοτήτων','τελείται','μασάς','περιφρούρησης','εισόδους','περιφρούρησης','αναπάντητα','υποκατάστημα','έκλεισαν','στηθεί','σοροί','επιβατών','προβλήματος','καραμπίνα','συμβολαίων','επισκεπτών','κουταβιών','ντου','διαδικτύου','διαδικτύου','κατηγορούσαν','υπέρογκη','υπέρογκα','κάμπινγκ','αναπάντητη','αποσταλεί','παραπόνου','αντίδικοι','παράξει','προκατόχους','συγκρίνει','σκανδάλων','κομματικό','πετάμε','κατάσβεση','θηλυ','επιχειρήσουν','κρίθηκε','κρούσματα','Παρασκευής','κατακαίει','αντιπάλους','πλήρωσε','λογισμικό','στενάχωρη','ασφάλισης','μέρας','Φυσικής','δεκάδων','διαμέτρου','τακτά','κομματικούς','κομματική','άνοιξη','σχολικής','βραδιά','γκέλα','ουραγό','ξαναπαίξει','συμφωνική','φορούσαν','Σωματείο','μπλακ','Προς','τουρνικέ','παρατάξεις','τραπεζιών','καταστήσει','κατάφεραν','κανένας','συνεργεία','συνεργείου','ακούσεις','ακούσεις','πατάνε','προτέρων','παραλίγο','τρίποντα','καρτών','μαστίχα','συμπεριέλαβε','μισό','μαστίχας','διατρέχει','συμπεριέλαβε','επίγειες','εντύπων','τετράμηνο','εγκαίρως','μικρόσωμα','μάγδα','μάγδα','συμμετείχε','καταλανικού','καταλανική','μωρέ','κονσερβοκουτι','μοίρασαν','αριστερας','σημερα','πρότερη','αυτοκριτική','διάφανα','Επικεφαλής','καραβάκι','βαρύτατο','κρέμασε','κρέμασε','μυθιστορήματος','αντιστοιχεί','Ποτέ','στεφάνου','επισκεφθεί','βασίσει','επισκεφθεί','Προσφύγων','Κάρολε','κρίνονται','πατάτες','πατάτες','μνήμα','μνήματα','τετράποδου','στεφάνων','ομότιτλο','φαντάρων','νυχτερινού','αδιάβλητο','καμπάνα','αδιάβλητο','Ανδρικάκη','περιβόητη','περιβόητη','περιβόητη','τετράποδοι','τόλμησαν','τόλμησαν','παλαιοί','κρατήσαμε','καταπάνω','συγκληθεί','διαδραση','διάβαζε','διάβαζα','διάβαζαν','Φυσική','Φυσικό','άνοδο','άνοδο','Εξάλλου','Κάμπου','φιλμ','φιλμ','φιλμ','φιλμ','καλούσες','κλόου','μπακ','δημοσιοποίησε','πέταξε','παντειο','καταβάσεις','κάθεται','Μέρες','παντελης','παράταξή','προβλήματος','επισκεφθεί','ζήτημα','ίδρυσε','έπαθαν','επιδόθηκε','αντιπάλου','προσεχή','κρίνουν','προβλήματά','επιλέξει','μέριμνα','καλούμε','εκατ','πρωτεύουσας','κομματικά','φαρμάκων','φαρμάκου','απέλυσε','προστατευόμενων','μαρτύρων','περιστύλιο','διάβασε','προστατευμένοι','χρονιάς','πετάνε','πολίτικη','ανατροφοδοτεί','καλούνται','πρωτογενή','ακριβών','διαφανεί','μαντάρα','Μπούλες','μοίρασε','αποκριάς','πολιτιστικούς','καλούμπα','Σφίγγει','αποκαλούμε','μαντάτα','σκόρερ','στεριάς','δίκτο','σοκάκι','καταστεί','διαβρώσει','σπείρεις','τελεί','κατάληψη','καθαρισμό','μαρίνα','παρών','κλιμάκια','προσφυγιάς','πλήρωσαν','εξάρθρωσε','εξάρθρωσε','παρατάσεις','πόρταλ','τόνισε','παραπανω','ακριβή','έκλεισα','κατάρτιση','λάβαμε','παραδοτέα','Επικρατείας','προβούν','πείσει','παραδόσεις','σπάσουν','χωριών','αντιπαροχή','Φυσικών','πολυτέκνων','αποδομήσω','προβλήθηκε','αποδομεί','Καθημερινής','Εσωτερικών','βαρειά','παραφράσει','ανακρίβεια','κρατήσει','παράταξης','μπαράζ','επικρίνει','πρόσφεραν','συνέλεξαν','μπουτόν','σκούρα','ανοιχτόχρωμο','εγκληματολόγος','εγκληματολόγο','περιστασιακή','παραλόγου','προγραμματισμένων','προβούμε','συνεργείο','λάβαρα','περσινό','συλληψή','σκιών','Παράταξη','χρίσμα','Συμμετείχαν','συμβάντων','ανεύρεση','σορός','μπάζα','ανακρίβειες','φαρμακερή','φαρμακερή','ασφάλιση','κατάπτυστος','κατάπτυστος','τετράποδος','μπαλάκια','πατριαρχείου','Πεντάγωνο','μήνυσης','προβλήµατα','προκαταβολική','κάρτ','νεύρα','φράγκο','ομολογιακό','ομολογιακού','μετρά','νομό','χρίσµα','Μαντάς','πλάτες','ρόδο','αποτέλεσαν','αποτέλεσαν','μήνυση','γυναικούλα','περσινή','νυχτερινής','δίνω','line','line','αντίποδα','Κομματικής','περιβόητου','περίδο','ακούσετε','ανήμερα','στημένη','στημένη','κομματικής','τόνιζε','μποι','Κόλλια','Παρασκάκης','σκάει','έκτροπα','επαναλάβει','Μπανάλ','αντιδημάρχου','κόλπους','κοντράρει','πρόσημο','κρατήσουν','καταληψιών','Πετσίτη','Πετσίτη','Ούτε','λιβάνι','Ποτάμηδες','ανάποδο','διορίες','φοράει','καταρράκωνε','τοσο','τοσο','κομματικοί','Περιστέρα','βαρελότα','προτού','δουλεια','κουτάκια','κουτάκια','προσέξτε','σκαλπ','σκαλπ','ζευγάς','σωματείο','μίλα','μούφα','εξοργισμένους','παράταξη','μπας','αρμάτων','αρμάτων','αρμάτων','παραδόθηκαν','Παράδεισο','Παράδεισο','απόλυτη','Δημοκράτες-66','ελιτ','καταρρεύσει','κομματίδια','κομματίδια','παρέλαβε','καμπανάκια','καμπάνες','μπούκαρε','αγωνιστής','τρέχα','προφυλαχθούν','υπουργάκου','κομματικού','στρατάρχη','στρατάρχη','μπουκάρουν','μπουκάρουν','περσινά','χωρίου','χωρίον','κατάφερες','κουμπάρες','γεράκος','πουλάει','πατάτα','χαλάσει','μπαλάκι','μπαλάκι','τουμπεκί','φαντασμάτων','Συμμετείχε','επικριτές','ψηφίζω','περίπολος','περίπολο','χωρίας','φυλακές','πολεμήσει','τερματίσει','χάρηκα','νόμιζες','γραμμένος','συνετέθη','χωρίστηκαν','επανακτήσει','ονομαστής','προφυλάκισης','καταλάβει','χαλίφη','αναστάσιμο','Ελευθέριος','προγραμματισμένες','προγραμματισμένη','τετράμηνη','προγραμματισμένο','αναπνευστικά','εξυπηρετούμενα','ανταλλάγματα','διευθύνοντος','συντηρημένες','σχηματισμένο','πλαγιές','υποδοχή','επισκεφθώ','προγραμματιστεί','χαρακτηριστεί','μπουλινγκ','κατάλαβες','μπόρα','επιχειρησιακή','οικονομολόγος','περίοπτη','σύμπλευση','δικαιώματα','συσχετισθεί','αστρονομικό','τροχός','πνευμόνια','κρεμάλες','κεφαλικό','καταλήψεις','κοινοτικές','μισά','συνεδριακό','πρωτοφανές','Ενοχος','αδιαφανές','φαντάζει','χρηματισμού','para','φαντάσματα','φίλες','πλευρό','επιμεληθεί','πρόσθεσε','επιτραπεί','οικισμούς','έπρεπε','μπάρα','πλαστικά','συσκευασίες','συσκευασία','συσκευασιών','σχεδίου','τραπεζικό','μέγεθός','σφουγγαράκια','επιχείρησαν','παράλια','συνεχίζει','επιχειρούσαν','προσθέτει','very','παρεκκλίσεις','πατάτε','διαρκές','κεράσουν','υποδοχής','συσχετισμός','υπόθεση.της','τροπή','εφόδου'] 

remove_words_no_tones = [remove_greek_tones(word) for word in remove_words]
custom_stopwords = custom_stopwords + remove_words_no_tones

# # Function to calculate Levenshtein distance
# def calculate_score(value,word):
#     print(word)
#     print(value)
#     limit_levenstein = int(len(word) / 2 + 1)
#     if word[0:limit_levenstein] == value[0:limit_levenstein]:
#         score = int(hamming(word, value))
#         if score < limit_levenstein - 1 and score != 0:
#             return score
#     return 0  # Return a large value for non-matching cases

# column=df_news.head(1)['cleaned_text_lemmas']
# words = column.split()

# Function to calculate Hamming distance between two strings
def hamming_distance(str1, str2):
    return hamming(np.array(list(str1)), np.array(list(str2)))



# In[148]:


# Function to find matches between article_content and Greek dictionary
def find_matches(article_content, dictionary):
    # Split article_content into words
    words_in_article = set(article_content.split())

    best_match_text_token = 100
    best_results_for_text_token = None
    results_for_text_token = []
    for word in dictionary:
        limit_levenstein = int(len(word) / 2 + 1)

        for article_word in words_in_article:
            score = 0
            best_result_for_text_token = 0
            if word[0:limit_levenstein] == article_word[0:limit_levenstein]:
                score = int(hamming(word, article_word))
                # print(article_word + ' ' + word + ' ' + str(score))
                if score < limit_levenstein - 1 and score != 0 and score < best_match_text_token:
                    best_match_text_token = score
                    print(article_word + ' ' + word)
                    best_result_for_text_token = word + " => " + article_word
                results_for_text_token.append(best_result_for_text_token)

        results_for_text_token.append(best_result_for_text_token)


# In[149]:




In [ ]:
df_news.columns

In [ ]:
df_news.shape

In [ ]:
df_news = df_news.rename(columns={'body': 'article_text','title': 'article_title', 'url':'article_url'})

In [ ]:
df_news = df_news.loc[df_news.article_text.apply(type) != float]

In [ ]:
df_news

In [ ]:
#first_article_matches = find_matches(df_news.iloc[2]['article_text'], df['Greek'])
rows_to_show = 1720

#calculate clean text
df_news[['cleanedtext','cleaned_text_lemmas']] =  df_news.head(rows_to_show)['article_text'].apply(lambda x: pd.Series(define_cleantext(x)))
df_news.loc[:,['cleanedtext','cleaned_text_lemmas']]
df_news.head(rows_to_show).loc[:,['cleanedtext','cleaned_text_lemmas']]


#define emotions
def define_emotions(column):
    words = column.split()

    anger = 0
    anticipation= 0
    disgust= 0
    fear = 0
    joy = 0
    negative = 0
    positive = 0
    sadness = 0
    surprise = 0
    trust = 0
    matched_tokens = 0

    for word in words:
        found_exact = False
        if word in indexes:
            found_exact = True
            indx = indexes[word]

            anger += df.at[indx, "anger"]
            anticipation += df.at[indx, "anticipation"]
            disgust += df.at[indx, "disgust"]
            fear += df.at[indx, "fear"]
            joy += df.at[indx, "joy"]
            negative += df.at[indx, "negative"]
            positive += df.at[indx, "positive"]
            sadness += df.at[indx, "sadness"]
            surprise += df.at[indx, "surprise"]
            trust += df.at[indx, "trust"]

            matched_tokens+= 1
    try:
        return (anger, anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust)

    except:
        return False


# In[158]:


#Load Emotions vocabulary
df = pd.read_csv("../LEXIKA/fixed_greek_EMOLEX_2023_01_sorted_no_english.csv")
indexes = {}
df = df.fillna("N/A")
for index, row in df.iterrows():
    df.at[index, "Greek"] = row["Greek"]
    indexes[df.at[index, "Greek"]] = index
#calculate emotions for lemmas
df_news[['anger',' anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']] =  df_news.head(rows_to_show)['cleaned_text_lemmas'].apply(lambda x: pd.Series(define_emotions(x)))
df_news[['anger',' anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']].head(rows_to_show)



# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the start and end times
print(f"Start Time: {start_time}")
print(f"End Time: {end_time}")
print(f"Elapsed Time: {elapsed_time} seconds")


#calculate emotions for cleaned text
df_news[['anger',' anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']] =  df_news.head(rows_to_show)['cleanedtext'].apply(lambda x: pd.Series(define_emotions(x)))
df_news[['anger',' anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']].head(rows_to_show)


# In[161]:


### TODO MERGE: emotions as there are  different exatmatches




In [ ]:
# In[151]:


#define valence
def define_valence(column):
    words = column.split()

    valence = 0.5
    arousal = 0.5
    dominance = 0.5
    matched_tokens = 0

    for word in words:
        found_exact = False
        if word in indexes:
            found_exact = True
            indx = indexes[word]

            valence = (valence + df.at[indx, "Valence"]) / 2
            arousal = (arousal + df.at[indx, "Arousal"]) / 2
            dominance = (dominance + df.at[indx, "Dominance"]) / 2

            matched_tokens+= 1
    try:
        return (valence, arousal,dominance)

    except:
        return False


# In[152]:


#Load Valence vocabulary
df = pd.read_csv("../LEXIKA/fixed_greek_NRC_VAD_lex_2023_01_sorted_no_english.csv")
indexes = {}
df = df.fillna("N/A")
for index, row in df.iterrows():
    df.at[index, "Greek"] = row["Greek"]
    indexes[df.at[index, "Greek"]] = index
#calculate valence for lemmas
df_news[['valence','arousal','dominance']] =  df_news.head(rows_to_show)['cleaned_text_lemmas'].apply(lambda x: pd.Series(define_valence(x)))


# In[153]:


df_news[['valence','arousal','dominance']].head(rows_to_show)


# In[154]:


#calculate valence for original
df_news[['valence','arousal','dominance']] =  df_news.head(rows_to_show)['cleanedtext'].apply(lambda x: pd.Series(define_valence(x)))


# In[155]:


df_news[['valence','arousal','dominance']].head(rows_to_show)


# In[156]:


### TODO MERGE: calculate  MEAN VALUES


In [ ]:
df_news[['valence','arousal','dominance']].head(rows_to_show)

In [ ]:

#define subjectivity
def define_subjectivity(column):
    words = column.split()

    strongsubj = 0
    weaksubj = 0
    matched_tokens = 0

    for word in words:
        found_exact = False
        if word in indexes:
            found_exact = True
            indx = indexes[word]

            if df.at[indx, "Strength"] == "strongsubj":
                strongsubj = strongsubj + 1
            elif df.at[indx, "Strength"] == "weaksubj":
                weaksubj = weaksubj + 1

            matched_tokens+= 1
    try:
        return (strongsubj, weaksubj)

    except:
        return False


# In[163]:


#Load Subjectivity vocabulary
df = pd.read_csv("../LEXIKA/Greek_Subjectivity_Fixed_2023_02.csv")
indexes = {}
df = df.fillna("N/A")
for index, row in df.iterrows():
    df.at[index, "Greek"] = row["Greek"]
    indexes[df.at[index, "Greek"]] = index
#calculate Subjectivity for lemmas
df_news[['strongsubj',' weaksubj']] =  df_news.head(rows_to_show)['cleaned_text_lemmas'].apply(lambda x: pd.Series(define_subjectivity(x)))


# In[164]:


df_news[['strongsubj',' weaksubj']].head(rows_to_show)


# In[165]:


#calculate subjectivity for cleaned text
df_news[['strongsubj',' weaksubj']] =  df_news.head(rows_to_show)['cleanedtext'].apply(lambda x: pd.Series(define_subjectivity(x)))
df_news[['strongsubj',' weaksubj']].head(rows_to_show)


# In[166]:


# TODO Merge the results


In [ ]:
nlp.add_pipe("textdescriptives/all")

In [ ]:
#Readability
#define readability
def define_readability(column):
    text = str(column)
    doc = nlp(text)
    output = td.extract_df(doc)
#     print(output[['flesch_reading_ease']])
#     output = output.drop(columns=['text'])
# output = output.drop(columns=['text','first_order_coherence',
#                               'second_order_coherence','flesch_reading_ease','flesch_kincaid_grade','smog','gunning_fog',
#                               'automated_readability_index','coleman_liau_index','lix','rix','passed_quality_check',
#                               'n_stop_words','alpha_ratio',
#                               'mean_word_length','doc_length','symbol_to_word_ratio_#','proportion_ellipsis','proportion_bullet_points',
#                               'contains_lorem ipsum','duplicate_line_chr_fraction','duplicate_paragraph_chr_fraction',
#                               'duplicate_ngram_chr_fraction_5','duplicate_ngram_chr_fraction_6','duplicate_ngram_chr_fraction_7',
#                               'duplicate_ngram_chr_fraction_8','duplicate_ngram_chr_fraction_9','duplicate_ngram_chr_fraction_10',
#                               'top_ngram_chr_fraction_2','top_ngram_chr_fraction_3','top_ngram_chr_fraction_4','oov_ratio','pos_prop_ADJ',
#                               'pos_prop_ADP','pos_prop_ADV','pos_prop_AUX','pos_prop_CCONJ','pos_prop_DET','pos_prop_INTJ','pos_prop_NOUN',
#                               'pos_prop_NUM','pos_prop_PART','pos_prop_PRON','pos_prop_PROPN','pos_prop_PUNCT','pos_prop_SCONJ','pos_prop_SYM',
#                               'pos_prop_VERB','pos_prop_X','entropy','perplexity','per_word_perplexity','token_length_mean','token_length_median',
#                               'token_length_std','sentence_length_mean','sentence_length_median','sentence_length_std','syllables_per_token_mean',
#                               'syllables_per_token_median','syllables_per_token_std','n_tokens','n_unique_tokens','proportion_unique_tokens',
#                               'n_characters','n_sentences','dependency_distance_mean','dependency_distance_std','prop_adjacent_dependency_relation_mean',
#                               'prop_adjacent_dependency_relation_std'])
#print(output.drop(columns=['text'])).to_json())
#     print(output)
#     print(output.to_dict())
#     print(output.columns)
    output_dict = output.to_dict()
    try:
        return (float((str(output_dict['flesch_reading_ease']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['flesch_kincaid_grade']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['smog']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['gunning_fog']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['n_characters']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['n_sentences']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['dependency_distance_mean']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['mean_word_length']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['n_stop_words']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['first_order_coherence']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['n_tokens']).split(':')[-1].rstrip('}'))),
                float((str(output_dict['n_unique_tokens']).split(':')[-1].rstrip('}'))))
    except:
        return False


In [ ]:
define_readability('μια προταση προσ αναλυση')

In [ ]:

#calculate readability for original text
df_news[['flesch_reading_ease','flesch_kincaid_grade','smog','gunning_fog','n_characters','n_sentences','dependency_distance_mean','mean_word_length','n_stop_words','first_order_coherence','n_tokens','n_unique_tokens']] =  df_news.head(rows_to_show)['article_text'].apply(lambda x: pd.Series(define_readability(x)))

df_news[['flesch_reading_ease','flesch_kincaid_grade','smog','gunning_fog','n_characters','n_sentences','dependency_distance_mean','mean_word_length','n_stop_words','first_order_coherence','n_tokens','n_unique_tokens']].head(rows_to_show)


# In[ ]:



In [ ]:
## DIVERSITY / SOURCES ####
def get_quotes_method_2(article_txt):
    
    if 'σύμφωνα με' in article_txt:
        article_txt.replace('σύμφωνα με', 'είπε')
    if 'Σύμφωνα με' in article_txt:
             article_txt.replace('Σύμφωνα με', 'είπε')
#             print('##### σύμφωνα με -> είπε')
    if 'έκανε λόγο' in article_txt:
             article_txt.replace('έκανε λόγο', 'είπε')
#             print('##### έκανε λόγο -> είπε')
    if 'είχε δηλώσει' in article_txt:
             article_txt.replace('είχε δηλώσει', 'είπε')
#             print('##### έκανε λόγο -> είπε')
    if 'σημειώνει ότι' in article_txt:
             article_txt.replace('σημειώνει ότι', 'είπε')
#             print('##### σημειώνει ότι -> είπε')            
    if 'λέει χαρακτηριστικά' in article_txt:
             article_txt.replace('λέει χαρακτηριστικά', 'είπε')
    
    doc = nlp(article_txt)
    
    total_sources = 0
    


    ## get quotes
    
    persons = set()
    persons = {'γραμματέας', 'δήμαρχος', 'υπουργός',  'πρόεδρος', 'κυβερνήτης', 'μάρτυρες', 
               'μάρτυρας', 'εφημερίδα', 'εκπρόσωπος', 'σύζυγος', 'γυναίκα' }
    
    for ent in doc.ents:
        if (ent.label_ == 'PERSON'):
          persons.update(ent.text.lower().split())
        

    
    sources_dictionary = ('σύμφωνα με','είπε', 'δήλωσε', 'υπογράμμισε', 'ανέφερε', 'ανέφεραν', 'πραγματοποίησε', 
                          'έλεγε', 'εξέφρασε', 'έγραψε', 'ενημέρωσε', 'τονίζει', 'αναφέρει', 'εξέδωσε', 'τόνισε', 'πρόσθεσε',
                         'εξηγεί', 'δηλώνει', 'λέει', 'αναφέρθηκε', 'επισήμανε')

    for sent in doc.sents:

        for possible_subject in sent:
            if (
                (possible_subject.dep_ in { 'nsubj', 'NOUN', 'PROPN', 'X'}) and 
                possible_subject.text.lower() in persons and
                possible_subject.head.pos_ == 'VERB' and 
                possible_subject.head.text.lower() in sources_dictionary
            ):
                total_sources+=1
                persons.remove(possible_subject.text.lower())
#                print("--------------->",possible_subject.head.text.lower(), "%%%%%%", possible_subject.text,"  ", 
#                      possible_subject.head.pos_, "***", possible_subject.pos_)
                break

    return total_sources

In [ ]:
df_news['sources'] = df_news.head(rows_to_show)['article_text'].apply(lambda x: get_quotes_method_2(x))

In [ ]:
df_news.head(rows_to_show)

In [ ]:
df_news.columns

In [ ]:
 df_news.shape[0]

In [ ]:
df_news.to_csv("../NEW_DATA/PENDING/nafteboriki_ellada_fb1.csv", index = False)